# Diabetes Machine Learning Group Project

Data Tables from 2017-18 from NHANES
- Diabetes (DIQ_J) https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/DIQ_J.htm
- Health Insurance (HIQ_J) https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/HIQ_J.htm
- Food Security (FSQ_J) https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/FSQ_J.htm
- Income (INQ_J) https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/INQ_J.htm

8,897 Original Respondents

## Merging

In [2]:
# Load packages
import pandas as pd

In [35]:
# Load in tables
diq = pd.read_csv("./resources/DIQ_J_2017-18.csv", index_col = 0)
fsq = pd.read_csv("./resources/FSQ_J_2017-18.csv", index_col = 0)
inq = pd.read_csv("./resources/INQ_J_2017-18.csv", index_col = 0)
hiq = pd.read_csv("./resources/HIQ_J_2017-18.csv", index_col = 0)
print(diq.head())
print(fsq.head())
print(inq.head())
print(hiq.head())

      SEQN  DIQ010  DID040  DIQ160  DIQ170  DIQ172  DIQ175A  DIQ175B  DIQ175C  \
1  93703.0       2     NaN     NaN     NaN     NaN      NaN      NaN      NaN   
2  93704.0       2     NaN     NaN     NaN     NaN      NaN      NaN      NaN   
3  93705.0       2     NaN     2.0     2.0     2.0      NaN      NaN      NaN   
4  93706.0       2     NaN     2.0     2.0     2.0      NaN      NaN      NaN   
5  93707.0       2     NaN     2.0     2.0     2.0      NaN      NaN      NaN   

   DIQ175D  ...  DIQ300D  DID310S  DID310D  DID320  DID330  DID341  DID350  \
1      NaN  ...      NaN      NaN      NaN     NaN     NaN     NaN     NaN   
2      NaN  ...      NaN      NaN      NaN     NaN     NaN     NaN     NaN   
3      NaN  ...      NaN      NaN      NaN     NaN     NaN     NaN     NaN   
4      NaN  ...      NaN      NaN      NaN     NaN     NaN     NaN     NaN   
5      NaN  ...      NaN      NaN      NaN     NaN     NaN     NaN     NaN   

   DIQ350U  DIQ360  DIQ080  
1      NaN     

In [36]:
# Isolate shared respondent IDs
diq_ids = diq["SEQN"].unique()
fsq_ids = fsq["SEQN"].unique()
inq_ids = fsq["SEQN"].unique()
hiq_ids = fsq["SEQN"].unique()
shared_ids = list(set(diq_ids) & set(fsq_ids) & set(inq_ids) & set(hiq_ids))

In [37]:
# Generate merged df of everything
merged = pd.merge(diq, fsq, on = "SEQN")
merged = pd.merge(merged, inq, on = "SEQN")
merged = pd.merge(merged, hiq, on = "SEQN")
merged.head()

,SEQN,DIQ010,DID040,DIQ160,DIQ170,DIQ172,DIQ175A,DIQ175B,DIQ175C,DIQ175D,...,HIQ031E,HIQ031F,HIQ031H,HIQ031I,HIQ031J,HIQ031AA,HIQ260,HIQ105,HIQ270,HIQ210
0,93703.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0
1,93704.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0
2,93705.0,2,NaN,2.0,2.0,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,2.0
3,93706.0,2,NaN,2.0,2.0,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0
4,93707.0,2,NaN,2.0,2.0,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0


In [41]:
# Export merged dataframe
merged.to_csv("./resources/merged.csv")

## Selection, Filtering, and Formatting

In [43]:
# Convert diabetic columns to intuitive labels


(8897, 132)